In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from functions import get_ohclv_df, get_fc_df

# 日频多因子数据路径
PATH_DAY_FREQ_FACTOR_DATA = 'data\\factor_mainforce\\'
# 日频股票OHCLV价格数据路径
PATH_DAY_FREQ_STOCK_DATA = 'data\\day_frequence_data\\'

N_COM = 3
WINDOW = 150
# 设置调仓频率f
FREQ = 30
TRAIN_PORTION = 0.8

# 获取因子组合数据
fc_df = get_fc_df(PATH_DAY_FREQ_FACTOR_DATA)
fc_df

fc_df is <class 'pandas.core.frame.DataFrame'>


,code,QUA,STD,SKEW,KURT,MTS,MTE,SR,time
0,000001.XSHE,0.145441,1362.454294,-0.380878,-0.638227,0.251744,0.095193,0.121012,2019-01-29
1,000002.XSHE,0.142786,1430.326218,-0.342911,-0.698062,0.236950,0.014028,0.083860,2019-01-29
2,000004.XSHE,0.012160,69.216894,0.582719,-0.856209,0.079915,0.010859,-0.038136,2019-01-29
3,000005.XSHE,0.064005,432.373110,-0.067846,-0.994510,0.082098,0.023157,0.217755,2019-01-29
4,000006.XSHE,0.053296,507.172876,0.082495,-0.984440,0.130100,0.030600,0.016365,2019-01-29
...,...,...,...,...,...,...,...,...,...
4667,688799.XSHG,0.087123,756.774780,0.262463,-0.964982,-0.003504,0.017580,-0.008545,2022-03-29
4668,688800.XSHG,0.239602,2937.989502,-1.051542,2.061847,0.029186,-0.038931,0.009075,2022-03-29
4669,688819.XSHG,0.150901,1235.037598,-0.258241,-0.809394,0.109766,-0.039530,-0.006641,2022-03-29
4670,688981.XSHG,0.202115,1711.586426,-0.509975,-0.501761,0.191117,0.011038,-0.003010,2022-03-29


In [2]:
price_df = get_ohclv_df(PATH_DAY_FREQ_STOCK_DATA)
price_df

price_df is <class 'pandas.core.frame.DataFrame'>


code,000001.XSHE,000002.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,000011.XSHE,...,688787.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688798.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
time,,,,,,,,,,,,,,,,,,,,,
2019-01-02,1079.71,3169.04,119.58,27.30,229.05,97.89,103.68,35.78,43.60,34.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,1090.29,3191.59,121.00,27.40,229.05,94.39,102.61,35.28,43.60,34.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-04,1145.51,3305.62,123.60,28.11,233.94,96.44,104.48,36.36,44.13,34.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-07,1144.33,3321.53,128.22,29.03,237.50,97.16,108.76,37.02,45.20,35.82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-08,1134.93,3314.90,125.17,28.93,236.61,94.63,110.63,37.61,44.80,35.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-22,1522.85,2809.30,67.39,18.09,219.60,95.48,61.83,126.01,44.80,42.52,...,51.77,57.57,166.77,36.44,95.40,31.63,139.55,37.07,38.45,38.91
2022-09-23,1522.84,2812.51,64.48,18.20,220.62,95.48,61.29,122.69,45.86,41.96,...,52.07,55.89,158.13,34.91,91.03,30.78,132.95,35.66,38.14,38.12
2022-09-26,1486.92,2801.27,62.10,17.28,214.54,92.22,59.68,120.68,44.00,40.51,...,50.22,53.37,155.80,36.56,95.42,30.46,136.40,36.34,37.47,38.60


In [3]:
fc_mean_dfg = fc_df[['time', 'QUA', 'STD', 'SKEW', 'KURT', 'MTE', 'SR']]
fc_mean_dfg = fc_mean_dfg.groupby(['time']).mean()
print(type(fc_mean_dfg))
fc_mean_dfg

<class 'pandas.core.frame.DataFrame'>


,QUA,STD,SKEW,KURT,MTE,SR
time,,,,,,
2019-01-30,0.089881,594.169019,-0.140669,-0.763749,0.022486,0.040221
2019-01-31,0.090049,596.527410,-0.141352,-0.763943,0.018227,0.037384
2019-02-01,0.089309,592.014104,-0.136241,-0.765151,0.016804,0.033003
2019-02-11,0.088529,587.624511,-0.131105,-0.768227,0.017045,0.032971
2019-02-12,0.088883,592.491004,-0.133817,-0.768220,0.017296,0.035229
...,...,...,...,...,...,...
2022-03-24,0.102183,552.385530,-0.199904,-0.704968,0.022813,0.037407
2022-03-25,0.102163,551.595571,-0.200140,-0.703880,0.021753,0.035606
2022-03-28,0.102211,549.869919,-0.201251,-0.702263,0.019841,0.036795


In [4]:
price_sum_df = price_df.sum(axis=1)
print(type(price_sum_df))
price_sum_df

<class 'pandas.core.series.Series'>


time
2019-01-02    269961.67
2019-01-03    267928.39
2019-01-04    274734.30
2019-01-07    279583.96
2019-01-08    279167.27
                ...    
2022-09-22    457746.50
2022-09-23    450770.95
2022-09-26    446415.03
2022-09-27    455579.90
2022-09-28    446857.32
Length: 910, dtype: float64

In [5]:
merge1 = price_sum_df.reset_index()
merge1['time'] = merge1['time'].astype('str')
merge2 = fc_mean_dfg.reset_index()
merge2['time'] = merge2['time'].astype('str')
all_data = pd.merge(merge1, merge2, on=['time'])
all_data.keys()
price_sum_df = all_data[['time', 0]]
fc_mean_dfg = all_data[['time', 'QUA', 'STD', 'SKEW', 'KURT', 'MTE', 'SR']]
price_sum_df = price_sum_df.set_index('time')
price_sum_df

,0
time,
2019-01-29,278429.84
2019-01-30,275061.07
2019-01-31,272582.08
2019-02-01,276123.50
2019-02-11,283058.29
...,...
2022-03-23,471747.89
2022-03-24,466707.11
2022-03-25,461715.64


In [6]:
fc_mean_dfg = fc_mean_dfg.set_index('time')
fc_mean_dfg


,QUA,STD,SKEW,KURT,MTE,SR
time,,,,,,
2019-01-29,0.090098,593.834685,-0.142117,-0.762891,0.025557,0.041431
2019-01-30,0.089881,594.169019,-0.140669,-0.763749,0.022486,0.040221
2019-01-31,0.090049,596.527410,-0.141352,-0.763943,0.018227,0.037384
2019-02-01,0.089309,592.014104,-0.136241,-0.765151,0.016804,0.033003
2019-02-11,0.088529,587.624511,-0.131105,-0.768227,0.017045,0.032971
...,...,...,...,...,...,...
2022-03-23,0.103399,558.029492,-0.206645,-0.703281,0.023424,0.030791
2022-03-24,0.102183,552.385530,-0.199904,-0.704968,0.022813,0.037407
2022-03-25,0.102163,551.595571,-0.200140,-0.703880,0.021753,0.035606


In [7]:
fc_np = np.array(fc_mean_dfg)

zs_fc_np = preprocessing.scale(fc_np)
print(f'before Z-score, fc_np.shape = {fc_np.shape}')
# print(fc_np)
print(f'before Z-score, zs_fc_np.shape = {zs_fc_np.shape}')
# print(zs_fc_np)

sc = MinMaxScaler()
mm_fc_np = sc.fit_transform(fc_np)
print(f'\nbefore Min-Max, fc_np.shape = {fc_np.shape}')
# print(fc_np)
print(f'before Min-Max, mm_fc_np.shape = {mm_fc_np.shape}')
# print(mm_fc_np)

before Z-score, fc_np.shape = (766, 6)
before Z-score, zs_fc_np.shape = (766, 6)

before Min-Max, fc_np.shape = (766, 6)
before Min-Max, mm_fc_np.shape = (766, 6)


In [8]:
price_np = np.array(price_sum_df)
# print(price_np)
return_rate_np = price_np.copy()
return_rate_np[0] = 0
begin_close_price = price_np[0]
for i in range(1, price_np.shape[0]):
    if i < FREQ:
        basic_price = begin_close_price
    else:
        basic_price = price_np[i - FREQ]
    return_rate_np[i] = (price_np[i] - basic_price) / basic_price
print(return_rate_np.shape)
# print(return_rate_np)

mm_price_np = sc.fit_transform(price_np)
print(f'\nbefore Min-Max, mm_price_np.shape = {mm_price_np.shape}')
# print(mm_price_np)
print(f'before Min-Max, mm_price_np.shape = {mm_price_np.shape}')
# print(mm_price_np)

(766, 1)

before Min-Max, mm_price_np.shape = (766, 1)
before Min-Max, mm_price_np.shape = (766, 1)


In [9]:
# 设置主成分个数n_com
print(f"PCA: n_components={N_COM}!")

PCA_MODEL = PCA(n_components=N_COM)
PCA_MODEL = PCA_MODEL.fit(mm_fc_np)
pc_np = PCA_MODEL.transform(mm_fc_np) #获取新矩阵

print(f'pc_np.shape = {pc_np.shape}')
print(pc_np)
print(f"PCA_MODEL.explained_variance_ratio_: {PCA_MODEL.explained_variance_ratio_}")

PCA: n_components=3!
pc_np.shape = (766, 3)
[[-0.37009953  0.1669541  -0.01893828]
 [-0.39605942  0.13341782 -0.00259018]
 [-0.42566951  0.07831266  0.02895568]
 ...
 [-0.19362676  0.1454647   0.25653435]
 [-0.19844313  0.13530539  0.26612832]
 [-0.39493826  0.06084374  0.47058675]]
PCA_MODEL.explained_variance_ratio_: [0.85602862 0.06713079 0.04658241]


In [10]:
# 设置滚动窗口大小w, 调仓频率f, 训练集占比tp
# f = 30 在更早的时候设置了调仓频率

print(f'pc_np.shape = {pc_np.shape}')
# print(pc_np)
print(f'return_rate_np.shape = {return_rate_np.shape}')
# print(return_rate_np)

# 建立自变量数据——w大小的滚动训练窗口的PC组合, 并且需要留空调仓频率长度末尾
X = []
for i in range(WINDOW, pc_np.shape[0] - FREQ):
    X.append(pc_np[i - WINDOW:i, :])
X = np.array(X)
# X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
print(f'X.shape = {X.shape}')
# print(X)

# 建立因变量数据
Y = []
# 将收益率作为因变量
# for i in range(w, return_rate_np.shape[0] - f):
#     Y.append(return_rate_np[i:i + f, :])
# 将价格作为因变量(需要修改，对价格进行归一化)
# for i in range(w, price_np.shape[0] - f):
#     Y.append(price_np[i:i + f, :])
# # 将归一化的价格作为因变量
for i in range(WINDOW, mm_price_np.shape[0] - FREQ):
    Y.append(mm_price_np[i:i + FREQ, :])

Y = np.array(Y)
Y = Y.reshape(Y.shape[0], Y.shape[1] * Y.shape[2])
print(f'Y.shape = {Y.shape}')
# print(Y)

# 划分数据集
X_train = X[0:int(X.shape[0] * TRAIN_PORTION)]
X_test = X[int(X.shape[0] * TRAIN_PORTION):]
Y_train = Y[0:int(Y.shape[0] * TRAIN_PORTION)]
Y_test = Y[int(Y.shape[0] * TRAIN_PORTION):]
print(
    f'X_train.shape={X_train.shape}\nX_test.shape={X_test.shape}\nY_train.shape={Y_train.shape}\nY_test.shape={Y_test.shape}'
)

pc_np.shape = (766, 3)
return_rate_np.shape = (766, 1)
X.shape = (586, 150, 3)
Y.shape = (586, 30)
X_train.shape=(468, 150, 3)
X_test.shape=(118, 150, 3)
Y_train.shape=(468, 30)
Y_test.shape=(118, 30)


In [11]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

X_train_LSTM= Variable(torch.Tensor(X_train))
X_test_LSTM= Variable(torch.Tensor(X_test))
Y_train_LSTM= Variable(torch.Tensor(Y_train))
Y_test_LSTM= Variable(torch.Tensor(Y_test))

device = torch.device(0)

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = self.init_hidden_lstm()

        self.fc = self.init_hidden()

    def init_hidden(self):
        return (nn.Linear(self.hidden_size,self.num_classes).to(device)).to(device)

    def init_hidden_lstm(self):
        return (nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size,
                            num_layers=self.num_layers, batch_first=True)).to(device)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size)).to(device)

        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))

        h_out = h_out.view(-1, self.hidden_size).to(device)

        out = self.fc(h_out).to(device)

        return out

# 设置模型参数
LR = 0.01

LSTM_MODEL = LSTM(num_classes=FREQ,
                  input_size=N_COM,
                  hidden_size=90,
                  num_layers=1,
                  seq_length=WINDOW)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(LSTM_MODEL.parameters(), lr=LR)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

LSTM_MODEL.to(device)
print(LSTM_MODEL)

mse = 0.1
# Train the model
for epoch in range(20000):
    trainX = X_train_LSTM.to(device)
    trainY = Y_train_LSTM.to(device)
    outputs = LSTM_MODEL(trainX)
    optimizer.zero_grad()

    # obtain the loss function
    loss = criterion(outputs, trainY)

    loss.backward()

    optimizer.step()
    if epoch % 1000 == 0:
        print("Epoch: %d, loss: %1.6f" % (epoch, loss.item()))
        torch.save(LSTM_MODEL, f'models/lstm_w{WINDOW}_f{FREQ}_tp{TRAIN_PORTION}_ncoms{N_COM}_mse_{"%.6f" % loss.item()}_epoch{epoch}.pth')

LSTM(
  (lstm): LSTM(3, 90, batch_first=True)
  (fc): Linear(in_features=90, out_features=30, bias=True)
)
Epoch: 0, loss: 0.434256
Epoch: 1000, loss: 0.001241
Epoch: 2000, loss: 0.000531
Epoch: 3000, loss: 0.000274
Epoch: 4000, loss: 0.000177
Epoch: 5000, loss: 0.001396
Epoch: 6000, loss: 0.000629
Epoch: 7000, loss: 0.000385
Epoch: 8000, loss: 0.002155
Epoch: 9000, loss: 0.000578
Epoch: 10000, loss: 0.000351
Epoch: 11000, loss: 0.000221
Epoch: 12000, loss: 0.000183
Epoch: 13000, loss: 0.000145
Epoch: 14000, loss: 0.000131
Epoch: 15000, loss: 0.000094
Epoch: 16000, loss: 0.000075
Epoch: 17000, loss: 0.000048
Epoch: 18000, loss: 0.000035
Epoch: 19000, loss: 0.000022
